In [1]:
import torch
import os
import random
import shutil
import torchvision
from torch import nn
from torch.nn import functional as F
from PIL import Image
import numpy as np
import time

In [2]:
batch_size = 32 # 批量大小
label_nums = 26 # 类别数量
Data_dir = "C:/Users/47925/Desktop/Data_sign_language" # 数据集路径
loss = nn.CrossEntropyLoss() # 交叉熵损失
lr = 1e-5 # 学习率
optimizer = torch.optim.Adam(net.parameters(),lr=lr) # 优化器
epoch = 1000 # 迭代次数

In [3]:
class Residual(nn.Module):
    # 残差块
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        # 前馈计算
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [4]:
b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [5]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [6]:
# ResNet18
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [7]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, label_nums))

In [8]:
def get_transform(is_train):
    # 预处理数据集
    if is_train:
        transform = torchvision.transforms.Compose([
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomRotation(degrees=(-45,45)),
            torchvision.transforms.ColorJitter(brightness=0.4,
                                               contrast=0.4,
                                               saturation=0.4),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                             [0.229, 0.224, 0.225])])
    else:
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize(224),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                             [0.229, 0.224, 0.225])])
    return transform

In [9]:
def build_dataset(is_train):
    # 使用torchvision更方便训练
    transform = get_transform(is_train)
    if is_train:
        path = os.path.join(Data_dir,"Train")
    else:
        path=os.path.join(Data_dir,"Valid")
    dataset = torchvision.datasets.ImageFolder(path,transform=transform)
    info = dataset.find_classes(path)
    print(info)
    return dataset

In [10]:
train_dataset = build_dataset(is_train=True)
valid_dataset = build_dataset(is_train=False)

(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25})
(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'], {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25})


In [11]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size,
                                                shuffle=True,
                                                drop_last=True)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset,
                                                batch_size,
                                                shuffle=False,
                                                drop_last=True)

In [12]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
net = net.to(device)

In [13]:
def init_weights(m):
    # 初始化权重
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)

In [14]:
net = net.apply(init_weights)

In [17]:
# net = torch.load("C://Users//47925//Desktop//Data_sign_language//logs//full_model179.pth") # 继续上次的训练

In [18]:
start_time = time.time()
# 开始训练
for i in range(epoch):
    print(f"第{i+1}轮训练")
    net.train()
    for j,(X,y) in enumerate(train_data_loader):
        optimizer.zero_grad()
        X,y = X.to(device),y.to(device)
        y_hat = net(X)
        l = loss(y_hat,y)
        l.backward()
        optimizer.step()
        train_acc = (y_hat.argmax(1)==y).sum()
        if j%100==0:
            print(f"训练时间：{time.time()-start_time}")
            print(f"第{j}个批次，loss：{l}，acc：{train_acc/batch_size}")
    
    # 在验证集上进行测试
    total_acc = 0
    with torch.no_grad():
        net.eval()
        for X,y in valid_data_loader:
            X,y = X.to(device),y.to(device)
            y_hat = net(X)
            acc = (y_hat.argmax(1)==y).sum()
            total_acc+=acc
    if (i+1)%10 == 0:
        torch.save(net, f'C:/Users/47925/Desktop/Data_sign_language/logs/full_model{i}.pth')
    
    print(f"整体验证集的acc：{total_acc/len(valid_data_loader.dataset)}")
        

第1轮训练
训练时间：0.10402655601501465
第0个批次，loss：0.0005105423042550683，acc：1.0
训练时间：13.509186029434204
第100个批次，loss：0.00013863760977983475，acc：1.0
训练时间：26.682159423828125
第200个批次，loss：0.014286128804087639，acc：1.0
训练时间：39.85335874557495
第300个批次，loss：0.007455478422343731，acc：1.0
训练时间：53.05134701728821
第400个批次，loss：0.0008073396747931838，acc：1.0
训练时间：66.258624792099
第500个批次，loss：0.01806616596877575，acc：1.0
训练时间：79.48361301422119
第600个批次，loss：0.0017764826770871878，acc：1.0
训练时间：92.71104979515076
第700个批次，loss：0.0012100773165002465，acc：1.0
训练时间：105.93252682685852
第800个批次，loss：0.0020640932489186525，acc：1.0
整体验证集的acc：0.9938461780548096
第2轮训练
训练时间：115.33464431762695
第0个批次，loss：7.82643910497427e-05，acc：1.0
训练时间：128.57163405418396
第100个批次，loss：0.02616819180548191，acc：0.96875
训练时间：141.82293844223022
第200个批次，loss：0.0020449631847441196，acc：1.0
训练时间：155.09000086784363
第300个批次，loss：0.0001863182260422036，acc：1.0
训练时间：168.35517024993896
第400个批次，loss：8.157407864928246e-05，acc：1.0
训练时间：181.63335371017456
第500个批次，l

第0个批次，loss：7.117289351299405e-05，acc：1.0
训练时间：1604.1926488876343
第100个批次，loss：0.002512203762307763，acc：1.0
训练时间：1622.3907589912415
第200个批次，loss：8.328563126269728e-05，acc：1.0
训练时间：1640.5152411460876
第300个批次，loss：0.00577498646453023，acc：1.0
训练时间：1657.9426884651184
第400个批次，loss：0.00011476421786937863，acc：1.0
训练时间：1671.5336391925812
第500个批次，loss：0.000507502700202167，acc：1.0
训练时间：1684.9866802692413
第600个批次，loss：0.08327274024486542，acc：0.96875
训练时间：1698.5476875305176
第700个批次，loss：5.886698636459187e-05，acc：1.0
训练时间：1712.0007348060608
第800个批次，loss：0.00013121584197506309，acc：1.0
整体验证集的acc：0.9938461780548096
第15轮训练
训练时间：1721.7193100452423
第0个批次，loss：0.0002736698661465198，acc：1.0
训练时间：1735.1936576366425
第100个批次，loss：0.00016726850299164653，acc：1.0
训练时间：1748.6936230659485
第200个批次，loss：0.00012076574057573453，acc：1.0
训练时间：1762.1971430778503
第300个批次，loss：0.0022612314205616713，acc：1.0
训练时间：1775.6272616386414
第400个批次，loss：8.486748265568167e-05，acc：1.0
训练时间：1789.0607233047485
第500个批次，loss：0.0168360695242

第0个批次，loss：0.0012874589301645756，acc：1.0
训练时间：3140.0152881145477
第100个批次，loss：0.0007043716032058001，acc：1.0
训练时间：3153.4479537010193
第200个批次，loss：0.00043669104343280196，acc：1.0
训练时间：3166.880141019821
第300个批次，loss：0.0007705349125899374，acc：1.0
训练时间：3180.310334920883
第400个批次，loss：0.0004682278377003968，acc：1.0
训练时间：3193.727536201477
第500个批次，loss：0.0004606911388691515，acc：1.0
训练时间：3207.178811311722
第600个批次，loss：0.03947019949555397，acc：0.96875
训练时间：3220.6102130413055
第700个批次，loss：0.01546352356672287，acc：1.0
训练时间：3234.0268471240997
第800个批次，loss：0.14786408841609955，acc：0.96875
整体验证集的acc：0.9925000071525574
第28轮训练
训练时间：3243.6520924568176
第0个批次，loss：0.0018960575107485056，acc：1.0
训练时间：3257.076639175415
第100个批次，loss：0.011735886335372925，acc：1.0
训练时间：3270.510722398758
第200个批次，loss：0.0030698170885443687，acc：1.0
训练时间：3283.959208726883
第300个批次，loss：0.0006078804144635797，acc：1.0
训练时间：3297.3933296203613
第400个批次，loss：2.651077556947712e-05，acc：1.0
训练时间：3310.825042247772
第500个批次，loss：0.0013035973533988，acc：

训练时间：4661.862091302872
第100个批次，loss：0.0004893938312307，acc：1.0
训练时间：4675.2942016124725
第200个批次，loss：0.0022194876801222563，acc：1.0
训练时间：4688.7111649513245
第300个批次，loss：0.00027400217368267477，acc：1.0
训练时间：4702.178187608719
第400个批次，loss：0.00015193417493719608，acc：1.0
训练时间：4715.611217260361
第500个批次，loss：0.0011453562183305621，acc：1.0
训练时间：4729.0442481040955
第600个批次，loss：0.0014125378802418709，acc：1.0
训练时间：4742.4623737335205
第700个批次，loss：0.0031774877570569515，acc：1.0
训练时间：4755.927410840988
第800个批次，loss：0.04578113928437233，acc：0.96875
整体验证集的acc：0.992307722568512
第41轮训练
训练时间：4765.600052595139
第0个批次，loss：0.0002033763303188607，acc：1.0
训练时间：4779.042776823044
第100个批次，loss：0.00589069165289402，acc：1.0
训练时间：4792.4610340595245
第200个批次，loss：0.00014652185200247914，acc：1.0
训练时间：4805.8924741744995
第300个批次，loss：0.000479242269648239，acc：1.0
训练时间：4819.325898170471
第400个批次，loss：0.0017009201692417264，acc：1.0
训练时间：4832.772933959961
第500个批次，loss：0.0014120076084509492，acc：1.0
训练时间：4846.224929094315
第600个批次，loss：0.

训练时间：6199.046786308289
第200个批次，loss：0.0020281870383769274，acc：1.0
训练时间：6212.506072759628
第300个批次，loss：0.0006919255829416215，acc：1.0
训练时间：6225.956276416779
第400个批次，loss：0.0004077463527210057，acc：1.0
训练时间：6239.39754986763
第500个批次，loss：0.023928433656692505，acc：1.0
训练时间：6252.843821048737
第600个批次，loss：0.00047784161870367825，acc：1.0
训练时间：6266.483976840973
第700个批次，loss：0.0006291915779002011，acc：1.0
训练时间：6280.023343324661
第800个批次，loss：0.0006281792302615941，acc：1.0
整体验证集的acc：0.9925000071525574
第54轮训练
训练时间：6289.844787836075
第0个批次，loss：0.014324424788355827，acc：1.0
训练时间：6303.354083538055
第100个批次，loss：0.002022204454988241，acc：1.0
训练时间：6317.037308931351
第200个批次，loss：0.0007515703910030425，acc：1.0
训练时间：6330.536913871765
第300个批次，loss：0.0003848181222565472，acc：1.0
训练时间：6344.092864990234
第400个批次，loss：0.027662673965096474，acc：0.96875
训练时间：6357.574856519699
第500个批次，loss：0.00046578014735132456，acc：1.0
训练时间：6371.09867978096
第600个批次，loss：0.00065270671620965，acc：1.0
训练时间：6384.571023464203
第700个批次，loss：0.000124

...

训练时间：18352.50179886818
第200个批次，loss：0.017154833301901817，acc：1.0
训练时间：18365.62776017189
第300个批次，loss：0.0014187635388225317，acc：1.0
训练时间：18378.772749185562
第400个批次，loss：0.0005609437357634306，acc：1.0
训练时间：18391.91073036194
第500个批次，loss：0.03759109601378441，acc：1.0
训练时间：18405.05770587921
第600个批次，loss：0.0007884141523391008，acc：1.0
训练时间：18418.19668507576
第700个批次，loss：0.0007893941947259009，acc：1.0
训练时间：18431.352437973022
第800个批次，loss：0.06900621205568314，acc：0.96875
整体验证集的acc：0.992307722568512
第158轮训练
训练时间：18440.81789803505
第0个批次，loss：0.055018939077854156，acc：0.96875
训练时间：18453.959799289703
第100个批次，loss：0.0003382040886208415，acc：1.0
训练时间：18467.116394519806
第200个批次，loss：0.0030452487990260124，acc：1.0
训练时间：18480.2757062912
第300个批次，loss：8.210770465666428e-05，acc：1.0
训练时间：18493.405142068863
第400个批次，loss：0.026318293064832687，acc：1.0
训练时间：18506.545117378235
第500个批次，loss：0.0011767102405428886，acc：1.0
训练时间：18519.679765939713
第600个批次，loss：0.0004187079903203994，acc：1.0
训练时间：18532.816739082336
第700个批次，los

训练时间：19832.036425352097
第100个批次，loss：9.895254333969206e-05，acc：1.0
训练时间：19845.159730672836
第200个批次，loss：0.005328567232936621，acc：1.0
训练时间：19858.290125131607
第300个批次，loss：0.02656685747206211，acc：1.0
训练时间：19871.452281475067
第400个批次，loss：0.000996389309875667，acc：1.0
训练时间：19884.59124827385
第500个批次，loss：9.639403288019821e-05，acc：1.0
训练时间：19897.72544836998
第600个批次，loss：0.00047378632007166743，acc：1.0
训练时间：19910.853411912918
第700个批次，loss：0.02342183329164982，acc：1.0
训练时间：19924.001371622086
第800个批次，loss：0.001556125353090465，acc：1.0
整体验证集的acc：0.9942307472229004
第171轮训练
训练时间：19933.479511260986
第0个批次，loss：0.015283457934856415，acc：1.0
训练时间：19946.621816158295
第100个批次，loss：0.00010456156451255083，acc：1.0
训练时间：19959.757235765457
第200个批次，loss：0.08496637642383575，acc：0.9375
训练时间：19973.18875002861
第300个批次，loss：0.0026999993715435266，acc：1.0
训练时间：19986.361719608307
第400个批次，loss：0.0005270112305879593，acc：1.0
训练时间：19999.496744394302
第500个批次，loss：0.01910848543047905，acc：1.0
训练时间：20012.642793655396
第600个批次，loss：

训练时间：21327.593539714813
第100个批次，loss：9.860376303549856e-05，acc：1.0
训练时间：21340.863539218903
第200个批次，loss：0.011435598134994507，acc：1.0
训练时间：21354.191805124283
第300个批次，loss：0.0014592281077057123，acc：1.0
训练时间：21367.354773283005
第400个批次，loss：0.00021565727365668863，acc：1.0
训练时间：21380.510548830032
第500个批次，loss：0.00510879373177886，acc：1.0
训练时间：21393.68653154373
第600个批次，loss：0.0013842424377799034，acc：1.0
训练时间：21406.833424568176
第700个批次，loss：0.001956529449671507，acc：1.0
训练时间：21419.989394187927
第800个批次，loss：0.00012516812421381474，acc：1.0
整体验证集的acc：0.993461549282074
第184轮训练
训练时间：21429.426526784897
第0个批次，loss：0.00482798321172595，acc：1.0
训练时间：21442.574507713318
第100个批次，loss：0.0005316248279996216，acc：1.0
训练时间：21455.80149126053
第200个批次，loss：0.0026968338061124086，acc：1.0
训练时间：21469.210525989532
第300个批次，loss：0.007643722463399172，acc：1.0
训练时间：21482.419506072998
第400个批次，loss：0.00015583554340992123，acc：1.0
训练时间：21495.624722003937
第500个批次，loss：0.00015220521891023964，acc：1.0
训练时间：21508.827107667923
第600个批次，l

第0个批次，loss：0.00041266149492003024，acc：1.0
训练时间：22824.038870573044
第100个批次，loss：0.00940394401550293，acc：1.0
训练时间：22837.223345518112
第200个批次，loss：0.04245109111070633，acc：0.96875
训练时间：22850.405335187912
第300个批次，loss：0.13715897500514984，acc：0.96875
训练时间：22863.592308998108
第400个批次，loss：7.667791214771569e-05，acc：1.0
训练时间：22876.790300369263
第500个批次，loss：6.856684194644913e-05，acc：1.0
训练时间：22889.967304229736
第600个批次，loss：7.092210580594838e-05，acc：1.0
训练时间：22903.148280382156
第700个批次，loss：0.00012423653970472515，acc：1.0
训练时间：22916.352523088455
第800个批次，loss：0.0002326188114238903，acc：1.0
整体验证集的acc：0.992884635925293
第197轮训练
训练时间：22925.81264925003
第0个批次，loss：0.0011425166158005595，acc：1.0
训练时间：22938.993630170822
第100个批次，loss：0.00039514951640740037，acc：1.0
训练时间：22952.203615665436
第200个批次，loss：0.0004408158711157739，acc：1.0
训练时间：22965.5679769516
第300个批次，loss：0.0003268510045018047，acc：1.0
训练时间：22978.785469532013
第400个批次，loss：0.0002593734534457326，acc：1.0
训练时间：22992.000494480133
第500个批次，loss：0.0005600185832

KeyboardInterrupt: 